# Thư viện

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import random


## 1. Cấu hình + Khởi tạo driver

In [ ]:
# Cấu hình dải trang cần crawl
START_PAGE = 348
END_PAGE = 400  # crawl từ 348 tới 400 (bao gồm)

CSV_PATH = "phongtro.csv"


def random_sleep(a=0.5, b=1.0):
    """
    Ngủ ngẫu nhiên trong khoảng [a, b] giây.
    (Tiện ích dùng chung)
    """
    time.sleep(random.uniform(a, b))


def create_driver():
    """
    Khởi tạo trình duyệt Chrome với các options cần thiết.
    (Thành viên 1 phụ trách)
    """
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # Bỏ comment để ẩn trình duyệt
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    )

    driver = webdriver.Chrome(options=chrome_options)
    return driver


## 2. Lấy danh sách link từng trang

In [ ]:
def get_links_for_page(driver, page):
    """
    Vào trang ?page=... và lấy tất cả link bài đăng.
    Trả về: list các URL bài đăng.
    """
    url = f"https://phongtro123.com/?page={page}"
    print(f"\nĐang crawl trang {page}...")
    driver.get(url)

    links = []

    try:
        # Đợi danh sách bài đăng load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul.post__listing li"))
        )
        random_sleep(0.5, 1.0)

        # Lấy tất cả link bài đăng 
        posts = driver.find_elements(By.CSS_SELECTOR, "h3.fs-6 a.line-clamp-2")
        links = [p.get_attribute("href") for p in posts if p.get_attribute("href")]

        print(f"Tìm thấy {len(links)} bài đăng")

        # Nếu không tìm thấy thì sẽ thử selector fallback
        if len(links) == 0:
            print("Không tìm thấy bài nào, thử selector khác...")
            posts = driver.find_elements(
                By.CSS_SELECTOR, "li.d-flex a[href*='phongtro123.com']"
            )
            raw_links = [
                p.get_attribute("href")
                for p in posts
                if p.get_attribute("href")
                and "/tinh-thanh/" not in p.get_attribute("href")
            ]
            links = list(set(raw_links))
            print(f"Fallback: Tìm thấy {len(links)} bài")
    except Exception as e:
        print(f"Lỗi khi load trang {page}: {e}")

    return links


## 3. Crawl thông tin cơ bản

In [ ]:
def extract_basic_info(driver, link, idx, total_in_page):
    """
    Vào 1 bài đăng và lấy các thông tin cơ bản:
    - title, address, price, area
    - ngaydang
    - description (mô tả)
    Trả về dict với các key trên + "url".
    """
    print(f"  [{idx}/{total_in_page}] Đang crawl: {link[:60]}...")
    driver.get(link)
    random_sleep(0.5, 1.0)

    # --- Lấy tiêu đề ---
    try:
        title = driver.find_element(By.TAG_NAME, "h1").text.strip()
    except:
        title = "N/A"

    # --- Lấy địa chỉ ---
    address = "N/A"
    try:
        # Cách 1: Tìm dòng có text "Địa chỉ:"
        address_row = driver.find_element(
            By.XPATH, "//td//div[contains(text(), 'Địa chỉ:')]"
        )
        address = (
            address_row.find_element(By.XPATH, "../..")
            .find_element(By.XPATH, "./td[2]")
            .text.strip()
        )
    except:
        try:
            # Cách 2: Fallback - dùng selector cũ
            elem = driver.find_element(By.XPATH, "//td[contains(text(), 'Đường')]")
            row = elem.find_element(By.XPATH, "..")
            addr_text = row.text.strip()
            addr_text = addr_text.replace("Đường:", "").strip()
            address = addr_text
        except:
            pass

    # --- Lấy giá ---
    try:
        price = driver.find_element(
            By.CSS_SELECTOR, "span.text-green.fs-5.fw-bold"
        ).text.strip()
    except:
        price = "N/A"

    # --- Lấy diện tích ---
    try:
        area = driver.find_element(
            By.CSS_SELECTOR,
            "div.justify-content-between > div.d-flex > span:nth-of-type(3)",
        ).text.strip()
    except:
        area = "N/A"

    # --- LẤY NGÀY ĐĂNG ---
    ngaydang = "N/A"
    try:
        ngaydang_row = driver.find_element(
            By.XPATH, "//td//div[contains(text(), 'Ngày đăng:')]"
        )
        time_element = (
            ngaydang_row.find_element(By.XPATH, "../..")
            .find_element(By.TAG_NAME, "time")
        )

        ngaydang_attr = time_element.get_attribute("title")
        if ngaydang_attr and ngaydang_attr.strip():
            ngaydang = ngaydang_attr.strip()
        else:
            ngaydang_text = time_element.text.strip()
            if ngaydang_text:
                ngaydang = ngaydang_text
    except:
        pass

    # --- LẤY THÔNG TIN MÔ TẢ ---
    description = "N/A"
    try:
        desc_container = driver.find_element(
            By.CSS_SELECTOR, "div.border-bottom.pb-3.mb-4"
        )
        paragraphs = desc_container.find_elements(By.TAG_NAME, "p")
        desc_lines = []

        for p in paragraphs:
            text = p.text.strip()
            if text:
                desc_lines.append(text)

        if desc_lines:
            description = " | ".join(desc_lines)

        if not description.strip():
            description = "N/A"
    except Exception as e:
        if idx == 1:
            print(f"Không lấy được mô tả: {e}")

    print(f"{title[:50]}...")

    basic_info = {
        "title": title,
        "address": address,
        "price": price,
        "area": area,
        "ngaydang": ngaydang,
        "thongtinmota": description,
        "url": link,
    }

    return basic_info


## 4. Crawl features + Ghép CSV + Vòng lặp chính

In [ ]:
def write_csv_header(writer):
    """
    Ghi dòng header vào file CSV.
    """



def run_crawler(driver, writer):
    """
    Vòng lặp chính:
    - Lặp qua các trang (gọi hàm thành viên 2)
    - Lặp qua các bài (gọi hàm thành viên 3 + 4)
    - Ghi vào CSV, đếm tổng số bài.
    """



## 5. Main

In [ ]:
def main():
    """
    Hàm main: tạo driver, mở file CSV, gọi run_crawler rồi đóng driver.
    """
    driver = create_driver()
    try:
        with open(CSV_PATH, "w", newline="", encoding="utf-8-sig") as csv_file:
            writer = csv.writer(csv_file)
            write_csv_header(writer)  
            total_rows = run_crawler(driver, writer)  

        print("\n" + "=" * 70)
        print(f"HOÀN TẤT! Đã ghi {total_rows} bài vào file '{CSV_PATH}'")
        print("=" * 70)
    except Exception as e:
        print(f"\nLỖI NGHIÊM TRỌNG: {e}")
    finally:
        driver.quit()
        print("Đã đóng trình duyệt")

In [ ]:
if __name__ == "__main__":
    main()
